In [ ]:
from flask import Flask, request, redirect,render_template,make_response
import pandas as pd
import pickle
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

app = Flask(__name__)


@app.route('/', methods=['GET','POST'])
def server():
    print("Connected")
    new_json = request.json
    #print(new_json["testDataECG"])
    #print("".join(request.json))
    json_string_ecg = new_json["testDataECG"]
    my_row_ecg = json_string_ecg.split("\n")
    #print(my_row)
    my_list_ecg=[]
    for x in my_row_ecg:
        temp=x.split(",")
        my_list_ecg.append(temp)
    #print(my_list)
    df1 = pd.DataFrame(my_list_ecg,columns=['MEAN_RR','MEDIAN_RR','SDRR','RMSSD','SDSD',	'SDRR_RMSSD',	'HR',	'pNN25',	'pNN50',	'SD1',	'SD2',	'KURT',	'SKEW',	'MEAN_REL_RR',	'MEDIAN_REL_RR',	'SDRR_REL_RR',	'RMSSD_REL_RR',	'SDSD_REL_RR',	'SDRR_RMSSD_REL_RR',	'KURT_REL_RR',	'SKEW_REL_RR',	'VLF',	'VLF_PCT',	'LF',	'LF_PCT',	'LF_NU',	'HF',	'HF_PCT',	'HF_NU',	'TP',	'LF_HF',	'HF_LF',	'sampen',	'higuci'])
    
    json_string_eeggsr = new_json["eegAndGSR"]
    my_row_eeggsr = json_string_eeggsr.split("\n")
    my_list_eeggsr=[]
    for x in my_row_eeggsr:
        temp=x.split(",")
        my_list_eeggsr.append(temp)
    #print(my_list)
    df2 = pd.DataFrame(my_list_eeggsr,columns=['FP2-F4','GSR'])
    #for y in my_list:
    #    print(y)
    #    df.append(y)
    print(df1)
    print(df2)
    #df1=df1.drop(columns='condition')
    #df1=df1.drop(columns='datasetId')
    #df1 = df1.to_numpy()
    #scaler = MinMaxScaler(feature_range=(0,1))
    #df1 = scaler.fit_transform(df1)
    
#     from sklearn.preprocessing import StandardScaler
#     sc = StandardScaler()
#     df2 = sc.fit_transform(df2)
    #df2.shape
    
    df1=df1.astype(float)
    df2=df2.astype(float)
    result_send = 0
    
    decisionTree = pickle.load( open("decisiontreeeegandgsr.sav", "rb" ) )
    print("Decision Tree")
    accuracyDT = decisionTree.predict(df2)
    print(accuracyDT)
    DTZero = 0
    DTOne = 0
    for result in accuracyDT:
        if result == 0:
            DTZero+=1
        else:
            DTOne+=1
    if DTZero>DTOne:
        print("Not Stressed")
    else:
        print("Stressed")
        result_send+=1
        
    decisionTreeECG = pickle.load( open("decisiontreeecg.sav", "rb" ) )
    print("Decision Tree ECG")
    accuracyDTECG = decisionTreeECG.predict(df1)
    print(accuracyDTECG)
    DTECGZero = 0
    DTECGOne = 0
    for result in accuracyDTECG:
        if result == "nostress":
            DTECGZero+=1
        else:
            DTECGOne+=1
    if DTECGZero>DTECGOne:
        print("Not Stressed")
    else:
        print("Stressed")
        result_send+=1
        
        
    #return make_response(request.args,200)
    '''
    aNNModel = pickle.load( open("annecg.sav", "rb" ) )
    print("ANN")
    accuracyaNN = aNNModel.predict(df1)
    print(accuracyaNN)
    aNNZero = 0
    aNNOne = 0
    for result in accuracyaNN:
        if result != "stress":
            aNNZero+=1
        else:
            aNNOne+=1
    if aNNZero>aNNOne:
        print("Not Stressed")
    else:
        print("Stressed")
    '''
    
    kNNModel = pickle.load( open("knnecg.sav", "rb" ) )
    print("KNN")
    accuracyKNN = kNNModel.predict(df1)
    print(accuracyKNN)
    KNNZero = 0
    KNNOne = 0
    for result in accuracyKNN:
        if result == "nostress":
            KNNZero+=1
        else:
            KNNOne+=1
    if KNNZero>KNNOne:
        print("Not Stressed")
    else:
        print("Stressed")
        result_send+=1
        
       
    randomForest = pickle.load( open("randomforesteegandgsr.sav", "rb" ) )
    print("Random Forest")
    accuracyRF = randomForest.predict(df2)
    print(accuracyRF)
    RFZero = 0
    RFOne = 0
    for result in accuracyRF:
        if result == 0:
            RFZero+=1
        else:
            RFOne+=1
    if RFZero>RFOne:
        print("Not Stressed")
    else:
        print("Stressed")
        result_send+=1
    
    
#     from sklearn.preprocessing import StandardScaler
#     sc = StandardScaler()
#     df2 = sc.fit_transform(df2)
#     df2.shape
    
    xgB1 = pickle.load( open("xgboosteegandgsr.sav", "rb" ) )
    print("XGBOOST")
    dtest = xgb.DMatrix(data = df2)
    accuracyXGB = xgB1.predict(dtest)
    print(accuracyXGB)
    
    XGBZero = 0
    XGBOne = 0
    for result in accuracyXGB:
        if result == 0:
            XGBZero+=1
        else:
            XGBOne+=1

    if XGBZero>XGBOne:
        print("Not Stressed")
    else:
        print("Stressed")
        result_send+=1
      
    if result_send > 2:
        print("STRESSED")
        return "1"
    else:
        print("NOT STRESSED")
        return "0"

if __name__ == '__main__':
    app.run(host='localhost')


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)


Connected
        MEAN_RR   MEDIAN_RR         SDRR        RMSSD         SDSD  \
0   939.4253707  948.357865  81.31774203  12.96443946  12.96419467   
1   898.1860468   907.00686  84.49723585  16.30527867   16.3052737   
2   814.2570213   827.52283  87.01445866   14.6322316  14.63127463   
3   848.6209148   851.57313  57.88633609  14.14265421  14.14254781   
4   772.7466067   775.00477  51.61021081  10.28816219  10.28809068   
5   776.4001781  775.153655   50.2057799  10.97499614  10.97486171   
6   810.5444307    822.2657   82.6692343  14.03395464  14.03395159   
7   793.6120423   720.74711  199.9621619  14.78600363  14.77937942   
8   994.5599086  1006.96165   107.655379  21.84425419  21.84419645   
9    775.171469  787.865115  90.13584792  13.42666905  13.42630064   
10  729.5015011   722.51064  80.94700886  23.53845167  23.53826612   
11  864.2132614  865.003415  92.35456926  21.99754357  21.99729942   
12  571.6601676  567.469775  46.25494741  8.850101059  8.850074886   
13  771.67

127.0.0.1 - - [12/Dec/2019 16:32:55] "POST / HTTP/1.1" 200 -


KNN
['stress' 'stress' 'stress' 'stress' 'stress' 'stress' 'stress' 'stress'
 'stress' 'stress' 'stress' 'stress' 'stress' 'stress' 'stress' 'stress'
 'stress' 'stress' 'stress' 'stress' 'stress' 'stress' 'stress' 'stress'
 'stress' 'stress' 'stress' 'stress' 'stress' 'stress']
Stressed
Random Forest
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Stressed
XGBOOST
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
Stressed
STRESSED
